## Basics 

### What is Runnable? 

* Langchain Expression Language is a way of grouping chains out of individual components known as runnables. 

* In more technical, Runnalbe is a **unit of work that can be invoked, batched, streamed, transformed and composed.**

* Runnable contains `invoke`,`stream` and `batch` methods. Also they have asyc like `ainvoke`, `astream`, and  `abatch`.




Each runnable will give different output, you should refer the documentation table [check here](https://python.langchain.com/v0.2/docs/concepts/#runnable-interface)


In [4]:
!pip install -U langchain-groq langchain --quiet

**Basic Chain Example**: 

In [11]:
api_key = "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"


#1st component 
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0.0,
    max_retries=2,
    api_key=api_key
)

#2nd component
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages([
    ("human","Could you suggest best {thing}")
])

#3rd component 
from langchain_core.output_parsers import StrOutputParser 
out = StrOutputParser()


#let's chain together everything, each component now acts as a runnable 
cute_chain = chat_template | llm | out 

output = cute_chain.invoke({"thing": "cycling"})
print(f"chain_output: {output}")

"I'd be happy to help you find the best cycling routes! To give you the most accurate suggestions, I'll need a bit more information. Here are some questions that can help me provide better recommendations:\n\n1. What is your location or the general area where you would like to go cycling?\n2. What is your experience level with cycling? (Beginner, intermediate, advanced)\n3. Do you prefer road cycling, mountain biking, or a mix of both?\n4. How long do you want to be cycling for? (A few hours, a full day, multiple days)\n5. Are there any specific points of interest or attractions you'd like to visit along the way?\n6. Do you have any preferences for the type of scenery you'd like to enjoy while cycling? (Coastal, forest, mountains, urban, etc.)\n\nOnce I have this information, I can help you find the best cycling routes that suit your needs and preferences."

## 1. Arguments of Runnables ( Batch, Stream, Invoke ) 

**Batch Example**: 

* In the previous example we had one input `thing`, let's say I want to give multiple inputs to `thing`, it's not possible, `invoke` method does not allow us to use multiple variables. 

* You can do one by one but we have a **better** approach called **batching**, it allows to run multiple invokes in parallel. 

* Use the batch if you want to use multiple inputs at once. 

In [ ]:

output = await cute_chain.abatch( [{"thing": "cycling"}, {"thing": "Apple mousd"}] )

output

["I'd be happy to help you find the best cycling routes! To give you the most accurate suggestions, I'll need a bit more information. Here are some questions that can help me provide better recommendations:\n\n1. What is your location or the general area where you would like to go cycling?\n2. What is your experience level with cycling? (Beginner, intermediate, advanced)\n3. Do you prefer road cycling, mountain biking, or a mix of both?\n4. How long do you want to be cycling for? (A few hours, a full day, multiple days)\n5. Are there any specific points of interest or attractions you'd like to visit along the way?\n6. Do you have any preferences for the type of scenery you'd like to enjoy while cycling? (Coastal, forest, mountains, urban, etc.)\n\nOnce I have this information, I can help you find the best cycling routes that suit your needs and preferences.",
 "I'm happy to help you find the best Apple mouse! When it comes to mice designed to work well with Apple computers, there are a

**Stream**: 

* Basically it's outputs a generator object. 

In [34]:

output = cute_chain.stream( {"thing": "cycling"} )

output

<generator object RunnableSequence.stream at 0x76b51b4d64d0>

In [35]:
for i in output: 
    print(i, end="")

I'd be happy to help you find the best cycling routes! To give you the most accurate suggestions, I'll need a bit more information. Here are some questions that can help me provide better recommendations:

1. What is your location or the general area where you would like to go cycling?
2. What is your experience level with cycling? (Beginner, intermediate, advanced)
3. Do you prefer road cycling, mountain biking, or a mix of both?
4. How long do you want to be cycling for? (A few hours, a full day, multiple days)
5. Are there any specific points of interest or attractions you'd like to visit along the way?
6. Do you have any preferences for the type of scenery you'd like to enjoy while cycling? (Coastal, forest, mountains, urban, etc.)

Once I have this information, I can help you find the best cycling routes that suit your needs and preferences.

### 2. Runnable & Runnable Sequence 


* Runnalbe is a **unit of work that can be invoked, batched, streamed, transformed and composed.**
* Runnalbe sequence is a sequence of runnables, where the output of each is input of next. 

Note!: Runnable & Runnable Sequence are a python class

In [36]:
type(cute_chain) # complete chain 

langchain_core.runnables.base.RunnableSequence

In [38]:
type(chat_template) # runnable ( if you go to the documentation, it's derived from runnable class )

langchain_core.prompts.chat.ChatPromptTemplate

### 3. Runnable Pass Through

* RunnalbePassThrough passes input through without alteration. 

In [43]:
from langchain_core.runnables import RunnablePassthrough

In [44]:
RunnablePassthrough().invoke([1, 2,3 ]) # simply passes the input 

[1, 2, 3]

In [45]:
#let's say we have two chains, 2nd chain will run based on 1st chain output 

idea_prompt = ChatPromptTemplate.from_messages([
    ("user", "you have to give a idea for this topic: {topic}")
])

action_prompt = ChatPromptTemplate.from_messages([
    ("user", "you have to make a action plan for this idea: {idea}")
])

idea_chain = idea_prompt | llm | StrOutputParser() 
action_chain = action_prompt | llm | StrOutputParser() 


In [49]:
idea = idea_chain.invoke({"topic": "how to implement complex chatbot"})
action = action_chain.invoke({"idea": idea})

print(action)

That's a great action plan for implementing a complex chatbot! Here's a bit more detail on each step:

1. Define the use case and goals: Start by identifying the problem your chatbot will solve and the goals it will help achieve. Consider the audience, the context, and the desired outcomes. This will help guide the design and development of the chatbot.
2. Choose a platform: Research and evaluate different chatbot development platforms based on your use case and goals. Consider factors such as ease of use, integrations, NLP capabilities, and cost. Some popular platforms include Dialogflow, Microsoft Bot Framework, and IBM Watson Assistant.
3. Design the conversation flow: Map out the conversation flow, including the different intents and entities the chatbot will need to recognize. Use a visual conversation flow design tool to help organize the flow and identify any gaps or areas for improvement.
4. Implement natural language processing (NLP): Choose a pre-built NLP engine or build a c

In [52]:
# Let's combine the two chains into one, The input for action_chain is a dictionary, so we should pass as a dictionary, let's create a dict and pass 

idea_and_action_chain = ( 
    idea_prompt 
    | llm 
    | StrOutputParser() 
    | {"idea": RunnablePassthrough()}
    | action_prompt 
    | llm 
    | StrOutputParser()
)

idea_and_action_chain.invoke({"topic": "how to implement complex chatbot using langchain?"})

"Sure, here's an action plan for implementing a complex chatbot using Langchain:\n\n1. Define the use case and functionality of your chatbot:\n\t* Identify the specific problems your chatbot will solve and the tasks it will perform.\n\t* Determine the types of questions your chatbot will answer and the information it will provide.\n\t* Consider the user experience and the conversational flow.\n2. Set up your Langchain environment:\n\t* Install Langchain and any necessary dependencies.\n\t* Configure your development environment.\n\t* Create a new Langchain project.\n3. Build the different components of your chatbot:\n\t* Natural Language Processing (NLP):\n\t\t+ Use Langchain's built-in NLP capabilities.\n\t\t+ Consider integrating with external NLP libraries or services for more advanced functionality.\n\t* Dialogue Management:\n\t\t+ Define the different states of the conversation.\n\t\t+ Use Langchain's dialog management system to define transitions between states.\n\t* Knowledge Ba

### 4. Visualize the Chains 

In [53]:
# To visualize this install this 
%pip install grandalf 

Note: you may need to restart the kernel to use updated packages.


In [54]:
idea_and_action_chain.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
  +--------------------+   
  | ChatPromptTemplate |   
  +--------------------+   
            *              
            *              
            *              
      +----------+         
      | ChatGroq |         
      +----------+         
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
     +-------------+       
     | Passthrough |       
     +-------------+       
            *              
            *              
            *              
  +--------------------+   
  | ChatPromptTemplate |   
  +--------------------+   
            *              
            *              
            *       

## 5. RunnableParallel 

In [55]:
from langchain_core.runnables import RunnableParallel

In [57]:
# consider we have two invocation to llm, that relies on same input ( read bellow )
chat_template_books = ChatPromptTemplate.from_messages([
    ("user", "Suggest three intermediate books for {programming_language}")
])
chat_template_course = ChatPromptTemplate.from_messages([
    ("user", "suggest three intermediate courses for {programming_language}")
])

# Both the prompts relies on same input and 2nd chain does not depends on the first chain. 
chain_books = chat_template_books | llm | StrOutputParser()
chain_course = chat_template_course | llm | StrOutputParser()

# Let's create a RunnalbeParallel 
chain_parallel = RunnableParallel({"first_chain": chain_books, "second_chain": chain_course})

output = chain_parallel.invoke({"programming_language": "python"})

In [58]:
print(output)

{'first_chain': 'Sure, here are three intermediate-level books for Python that I would recommend:\n\n1. "Python Cookbook" by David Beazley and Brian K. Jones: This book is a collection of recipes for solving common programming tasks using Python. It covers a wide range of topics, including data structures, file I/O, network programming, concurrency, and metaprogramming. The recipes are presented in a clear and concise format, making it easy to find and implement solutions to specific problems.\n2. "Fluent Python" by Luciano Ramalho: This book is a comprehensive guide to mastering Python\'s advanced features. It covers topics such as data model customization, decorators, context managers, generators, and coroutines. The book is written in a clear and engaging style, with plenty of examples and exercises to help readers deepen their understanding of the material.\n3. "Effective Python" by Brett Slatkin: This book is a collection of 59 specific ways to write better Python code. It covers 

In [59]:
chain_parallel.get_graph().print_ascii()

      +-----------------------------------------+        
      | Parallel<first_chain,second_chain>Input |        
      +-----------------------------------------+        
                   **               **                   
                ***                   ***                
              **                         **              
+--------------------+            +--------------------+ 
| ChatPromptTemplate |            | ChatPromptTemplate | 
+--------------------+            +--------------------+ 
           *                                 *           
           *                                 *           
           *                                 *           
     +----------+                      +----------+      
     | ChatGroq |                      | ChatGroq |      
     +----------+                      +----------+      
           *                                 *           
           *                                 *           
           *  

In [61]:
# After getting all the output ( two chains ), I need to pass both the output to another invocation for time estimation, let's see how to do 
chat_template_books = ChatPromptTemplate.from_messages([
    ("user", "Suggest three intermediate books for {programming_language}")
])
chat_template_course = ChatPromptTemplate.from_messages([
    ("user", "suggest three intermediate courses for {programming_language}")
])
chat_template_final_estimation = ChatPromptTemplate.from_messages([
    ("user", """You will get two inputs \n1. books \n2.Course \n
                You have to estimate how long will it take to complete, just estimte the answer\n 
                Here is the books: {books} and courses: {courses}""")
]) 

#The last chat_prompt will take the two outputs from preceding chains and do the estimation. 

chain_books = chat_template_books | llm | StrOutputParser()
chain_course = chat_template_course | llm | StrOutputParser() 
final_estimation = chat_template_final_estimation | llm | StrOutputParser() 


final_chain = (
    RunnableParallel({"books": chain_books, "courses": chain_course})
    | final_estimation 
    | llm 
    | StrOutputParser()
)

final_output = final_chain.invoke({"programming_language": "python"})

In [62]:
print(final_output)

Thank you for the estimates. Based on my current schedule, I believe it will take me closer to the longer end of the ranges you provided to complete each book and course. I appreciate the advice to focus on understanding the material rather than rushing through it, and I will keep that in mind as I work through these resources.

In addition to the resources you listed, I am also interested in learning more about web development with Python. Do you have any recommendations for books or courses on this topic?

Here are some recommendations for web development with Python:

* "Web Development with Python" by Michael Herman: This book covers the basics of web development with Python, including HTML, CSS, JavaScript, and web frameworks like Flask and Django. It's a great resource for beginners.
* "Flask Web Development" by Miguel Grinberg: This book focuses on Flask, a lightweight web framework for Python. It covers topics like routing, templates, forms, and databases.
* "Django for Profess

In [63]:
final_chain.get_graph().print_ascii()

            +------------------------------+             
            | Parallel<books,courses>Input |             
            +------------------------------+             
                   **               **                   
                ***                   ***                
              **                         **              
+--------------------+            +--------------------+ 
| ChatPromptTemplate |            | ChatPromptTemplate | 
+--------------------+            +--------------------+ 
           *                                 *           
           *                                 *           
           *                                 *           
     +----------+                      +----------+      
     | ChatGroq |                      | ChatGroq |      
     +----------+                      +----------+      
           *                                 *           
           *                                 *           
           *  

## 6. RunnalbeLambda 

* Converts normal python functions into runnalbe 

In [68]:
# Let's create two simple lambda function 

find_sum = lambda x: sum(x)
find_sq = lambda x: x**2

# We can't pipe this in our chain, becaure it aren't runnalbe object, two convert to runnable object, we use RunnableLambda 
from langchain_core.runnables import RunnableLambda 

runnalbe_sum = RunnableLambda(find_sum)   #You can also create a function ans pass it here 
runnalbe_sq = RunnableLambda(find_sq)

# print(runnalbe_sum.invoke([1, 2, 3])) ## You will get the same output as before 
lambda_runnalbe = runnalbe_sum | runnalbe_sq 

print(lambda_runnalbe.invoke([1, 2, 3]))

36


In [69]:
lambda_runnalbe.get_graph().print_ascii()

+-------------+  
| LambdaInput |  
+-------------+  
        *        
        *        
        *        
   +--------+    
   | Lambda |    
   +--------+    
        *        
        *        
        *        
   +--------+    
   | Lambda |    
   +--------+    
        *        
        *        
        *        
+--------------+ 
| LambdaOutput | 
+--------------+ 


In [71]:
#you can also create a runnables using chains 
from langchain_core.runnables import chain

@chain
def runnable_multiple(x): 
    return x*x 

new_chain = lambda_runnalbe | runnable_multiple 

print(new_chain.invoke([1, 2, 3]))

1296


## 7. Assign in RunnablePassThrough

In [83]:
# It allows us to include additional keys to the dictionary we are passing through 
# If you want to get some information from the output / input of some other chain, use this,

RunnablePassthrough().invoke({"input": "hi"}) # this is a normal function, but I want [h] in [first letter], [i] in [second letter] key

grab_first_letter = lambda x: x['input'][0] #input is the key in our RunnalbePassthrough 
grab_second_letter = lambda x: x['input'][1]

RunnablePassthrough.assign(first_letter=grab_first_letter, second_letter=grab_second_letter).invoke({'input': "hi"})

{'input': 'hi', 'first_letter': 'h', 'second_letter': 'i'}

## 9. ItemGetter 

In [86]:
from operator import itemgetter

# It tries to retrieve the item in the given object, if the object supports itemgetter thunder method 

itemgetter(0)("hi")
itemgetter('hello')({"hello": "hi"})
itemgetter(2)([1,2,3])

3

In [89]:
item_getter = RunnableLambda(itemgetter("chat_history")).invoke({"chat_history": "how are you darling"})

## 9. Adding Multiple Runnalbes ( Example Real world )

In [122]:
# Let's add memory to our llms using LCEL while using all we learnt 

from langchain import PromptTemplate
from langchain.memory import ConversationSummaryMemory 

template_for_answer = """
You're a helpful AI Assistant, based on the given chat_hisotry you have to answer, if you don't know just say I don't know. 

conversation history: 
{chat_history}

Human: 
{question}
"""

template_for_answer = PromptTemplate.from_template(template_for_answer)
memory = ConversationSummaryMemory(llm=llm, memory_key="chat_history")


## Rather than immediately creating a chain, let's invoke one by one and build at last
final_chain = (
    RunnablePassthrough.assign(  chat_history=RunnableLambda(memory.load_memory_variables) | RunnableLambda(itemgetter("chat_history")) )
    | template_for_answer
    | llm 
    | StrOutputParser() 
    )


In [120]:
question = "Can you give me the intersting fact?"

In [124]:
out = final_chain.invoke({"question": question})
memory.save_context(inputs= {"input": question}, outputs ={"output": out})

In [127]:
@chain
def final_chain(question): 

    final_chain = (
        RunnablePassthrough.assign(  chat_history=RunnableLambda(memory.load_memory_variables) | RunnableLambda(itemgetter("chat_history")) )
        | template_for_answer
        | llm 
        | StrOutputParser() 
        )
    
    out = final_chain.invoke({"question": question})
    memory.save_context(inputs= {"input": question}, outputs ={"output": out})

    return out 


In [131]:
final_chain.invoke("What's my naem?")


"Based on our conversation history, your name is Aravind. I'm here to help answer your questions to the best of my ability. If I don't know something, I'll let you know. Is there something specific you'd like to know about octopuses or any other topic?"